<a href="https://colab.research.google.com/github/Marryele/pyspark/blob/main/Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##tutorial 1 e 2

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 33 kB/s 
     |████████████████████████████████| 199 kB 64.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a05d138473b08824fb640443f89c4a4ea2d7d7b79c2d62de3efdfbe7d5ceddbe
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Test').getOrCreate()

In [ ]:
spark

In [ ]:
#Carregar um arquivo com spark  .json .parquet    
df = spark.read.csv('/content/file.csv', sep=';') 

In [ ]:
#Exibir o dataframe
df.show() 

+-----+----+----+
|  _c0| _c1| _c2|
+-----+----+----+
| Name| Age|City|
|Maria|  24|  BH|
| Jose|  29|  RJ|
| Joao|  21|  SP|
| null|null|null|
+-----+----+----+



In [ ]:
#Passando Option, podemos pegar o header do arquivo 
spark.read.option('header','true').csv('/content/file.csv', sep=';').show()

+-----+----+----+
| Name| Age|City|
+-----+----+----+
|Maria|  24|  BH|
| Jose|  29|  RJ|
| Joao|  21|  SP|
| null|null|null|
+-----+----+----+



In [ ]:
df_spark = spark.read.option('header','true').csv('/content/file.csv', sep=';')

In [ ]:
type(df_spark)

pyspark.sql.dataframe.DataFrame

In [ ]:
df_spark.head()

Row(Name='Maria', Age='24', City='BH')

In [ ]:
# ver o esquema
df_spark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- City: string (nullable = true)



In [ ]:
# adicionando o inferSchema para pegar os types corretos
df_spark = spark.read.option('header','true').csv('/content/file.csv', sep=';',inferSchema=True)

In [ ]:
# ver o esquema
df_spark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- City: string (nullable = true)



In [ ]:
# adicionando o headr no read em vez de options
df_spark = spark.read.csv('/content/file.csv', sep=';',inferSchema=True,header=True)

In [ ]:
df_spark.show()

+-----+----+----+
| Name| Age|City|
+-----+----+----+
|Maria|  24|  BH|
| Jose|  29|  RJ|
| Joao|  21|  SP|
| null|null|null|
+-----+----+----+



In [ ]:
df_spark.columns

['Name', 'Age', 'City']

In [ ]:
#selecionando uma coluna
df_spark.select('Name').show()

+-----+
| Name|
+-----+
|Maria|
| Jose|
| Joao|
| null|
+-----+



In [ ]:
#selecionando mais de uma coluna
df_spark.select(['Name','Age']).show()

+-----+----+
| Name| Age|
+-----+----+
|Maria|  24|
| Jose|  29|
| Joao|  21|
| null|null|
+-----+----+



In [ ]:
df_spark['Name']

Column<'Name'>

In [ ]:
#ver os data types
df_spark.dtypes

[('Name', 'string'), ('Age', 'int'), ('City', 'string')]

In [ ]:
df_spark.describe()

DataFrame[summary: string, Name: string, Age: string, City: string]

In [ ]:
df_spark.describe().show()

+-------+-----+------------------+----+
|summary| Name|               Age|City|
+-------+-----+------------------+----+
|  count|    3|                 3|   3|
|   mean| null|24.666666666666668|null|
| stddev| null| 4.041451884327381|null|
|    min| Joao|                21|  BH|
|    max|Maria|                29|  SP|
+-------+-----+------------------+----+



In [ ]:
#adicionando colunas
df_spark.withColumn('Age after 2 years', df_spark['Age']+2)

DataFrame[Name: string, Age: int, City: string, Age after 2 years: int]

In [ ]:
df_spark = df_spark.withColumn('Age after 2 years', df_spark['Age']+2)

In [ ]:
df_spark.show()

+-----+----+----+-----------------+
| Name| Age|City|Age after 2 years|
+-----+----+----+-----------------+
|Maria|  24|  BH|               26|
| Jose|  29|  RJ|               31|
| Joao|  21|  SP|               23|
| null|null|null|             null|
+-----+----+----+-----------------+



In [ ]:
#removendo colunas
df_spark.drop('Age after 2 years').show()  #não é inplance, preciso atribuir igual python

+-----+----+----+
| Name| Age|City|
+-----+----+----+
|Maria|  24|  BH|
| Jose|  29|  RJ|
| Joao|  21|  SP|
| null|null|null|
+-----+----+----+



In [ ]:
#renomeando colunas
df_spark.withColumnRenamed('Name','New Name').show()

+--------+----+----+-----------------+
|New Name| Age|City|Age after 2 years|
+--------+----+----+-----------------+
|   Maria|  24|  BH|               26|
|    Jose|  29|  RJ|               31|
|    Joao|  21|  SP|               23|
|    null|null|null|             null|
+--------+----+----+-----------------+



##tutorial 3

Como tratar valores nulos

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('test').getOrCreate()

In [4]:
#Passando Option, podemos pegar o header do arquivo 
spark.read.option('header','true').csv('/content/teste2.csv', sep=';',inferSchema=True).show()

+------+----+----------+------+
|  Name| age|experience|salary|
+------+----+----------+------+
|  Kate|  31|        10| 30000|
| Susan|  30|         8| 25000|
|   Sun|  29|         4| 20000|
|  Paul|  24|         3| 20000|
|Hailey|  21|         1| 15000|
| Shein|  23|         2| 18000|
|Marcus|null|      null| 40000|
|  null|  34|        10| 38000|
|  null|  36|      null|  null|
+------+----+----------+------+



In [5]:
#Passando Option, podemos pegar o header do arquivo 
df_pyspark = spark.read.option('header','true').csv('/content/teste2.csv', sep=';',inferSchema=True)

In [6]:
#removendo linhas específicas, baseadas em valores nulos
df_pyspark.na.drop().show()

+------+---+----------+------+
|  Name|age|experience|salary|
+------+---+----------+------+
|  Kate| 31|        10| 30000|
| Susan| 30|         8| 25000|
|   Sun| 29|         4| 20000|
|  Paul| 24|         3| 20000|
|Hailey| 21|         1| 15000|
| Shein| 23|         2| 18000|
+------+---+----------+------+



In [7]:
## how='any' = linhas com qualquer valor nulo
## how='all' = linhas com todos valores nulos
df_pyspark.na.drop(how='all').show()

+------+----+----------+------+
|  Name| age|experience|salary|
+------+----+----------+------+
|  Kate|  31|        10| 30000|
| Susan|  30|         8| 25000|
|   Sun|  29|         4| 20000|
|  Paul|  24|         3| 20000|
|Hailey|  21|         1| 15000|
| Shein|  23|         2| 18000|
|Marcus|null|      null| 40000|
|  null|  34|        10| 38000|
|  null|  36|      null|  null|
+------+----+----------+------+



In [18]:
# subset = ['nome das colunas']
df_pyspark.na.drop(how='all',subset=['Name']).show()

+------+----+----------+------+
|  Name| age|experience|salary|
+------+----+----------+------+
|  Kate|  31|        10| 30000|
| Susan|  30|         8| 25000|
|   Sun|  29|         4| 20000|
|  Paul|  24|         3| 20000|
|Hailey|  21|         1| 15000|
| Shein|  23|         2| 18000|
|Marcus|null|      null| 40000|
+------+----+----------+------+



In [22]:
# Preenchendo valores nulos
df_pyspark.na.fill(0,['age','experience']).show() #[nome das colunas]

+------+---+----------+------+
|  Name|age|experience|salary|
+------+---+----------+------+
|  Kate| 31|        10| 30000|
| Susan| 30|         8| 25000|
|   Sun| 29|         4| 20000|
|  Paul| 24|         3| 20000|
|Hailey| 21|         1| 15000|
| Shein| 23|         2| 18000|
|Marcus|  0|         0| 40000|
|  null| 34|        10| 38000|
|  null| 36|         0|  null|
+------+---+----------+------+



In [23]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['age','experience','salary'],
    outputCols=["{}_imputed".format(c) for c in ['age','experience','salary']]
).setStrategy("mean")

In [24]:
#preenchi os valores nulos com a média dos valores das outras colunas

imputer.fit(df_pyspark).transform(df_pyspark).show()

+------+----+----------+------+-----------+------------------+--------------+
|  Name| age|experience|salary|age_imputed|experience_imputed|salary_imputed|
+------+----+----------+------+-----------+------------------+--------------+
|  Kate|  31|        10| 30000|         31|                10|         30000|
| Susan|  30|         8| 25000|         30|                 8|         25000|
|   Sun|  29|         4| 20000|         29|                 4|         20000|
|  Paul|  24|         3| 20000|         24|                 3|         20000|
|Hailey|  21|         1| 15000|         21|                 1|         15000|
| Shein|  23|         2| 18000|         23|                 2|         18000|
|Marcus|null|      null| 40000|         28|                 5|         40000|
|  null|  34|        10| 38000|         34|                10|         38000|
|  null|  36|      null|  null|         36|                 5|         25750|
+------+----+----------+------+-----------+------------------+--

##tutorial 4

Filter = >=  <=  and or

In [26]:
#Passando Option, podemos pegar o header do arquivo 
df_pyspark = spark.read.option('header','true').csv('/content/teste2.csv', sep=';',inferSchema=True)

#removendo nulos
df_pyspark = df_pyspark.na.drop()

#exibindo
df_pyspark.show()

+------+---+----------+------+
|  Name|age|experience|salary|
+------+---+----------+------+
|  Kate| 31|        10| 30000|
| Susan| 30|         8| 25000|
|   Sun| 29|         4| 20000|
|  Paul| 24|         3| 20000|
|Hailey| 21|         1| 15000|
| Shein| 23|         2| 18000|
+------+---+----------+------+



In [27]:
## Operações com filtros 

# Filtrar salario menores ou igual a 20000

df_pyspark.filter("salary <= 20000").show()

+------+---+----------+------+
|  Name|age|experience|salary|
+------+---+----------+------+
|   Sun| 29|         4| 20000|
|  Paul| 24|         3| 20000|
|Hailey| 21|         1| 15000|
| Shein| 23|         2| 18000|
+------+---+----------+------+



In [28]:
#filtrando e selecionando colunas
df_pyspark.filter("salary <= 20000").select(['Name','age']).show()

+------+---+
|  Name|age|
+------+---+
|   Sun| 29|
|  Paul| 24|
|Hailey| 21|
| Shein| 23|
+------+---+



In [30]:
#outro jeito de filtrar
df_pyspark.filter(df_pyspark['salary'] <= 20000).show()

+------+---+----------+------+
|  Name|age|experience|salary|
+------+---+----------+------+
|   Sun| 29|         4| 20000|
|  Paul| 24|         3| 20000|
|Hailey| 21|         1| 15000|
| Shein| 23|         2| 18000|
+------+---+----------+------+



In [34]:
# mais de 1 condição dentro do filter
df_pyspark.filter((df_pyspark['salary'] <= 20000) & # ou é o simbolo: |   
                  (df_pyspark['age'] > 23 )).show()

+----+---+----------+------+
|Name|age|experience|salary|
+----+---+----------+------+
| Sun| 29|         4| 20000|
|Paul| 24|         3| 20000|
+----+---+----------+------+



In [35]:
# negando o filtro com ~ 
df_pyspark.filter(~(df_pyspark['salary'] <= 20000)).show()

+-----+---+----------+------+
| Name|age|experience|salary|
+-----+---+----------+------+
| Kate| 31|        10| 30000|
|Susan| 30|         8| 25000|
+-----+---+----------+------+



##Tutorial 5 

Groupby e aggregated functions


In [4]:
#Passando Option, podemos pegar o header do arquivo 
df_pyspark = spark.read.option('header','true').csv('/content/teste2.csv', sep=';',inferSchema=True)

#removendo nulos
df_pyspark = df_pyspark.na.drop()

#exibindo
df_pyspark.show()

+------+---+----------+------+
|  Name|age|experience|salary|
+------+---+----------+------+
|  Kate| 31|        10| 30000|
| Susan| 30|         8| 25000|
|   Sun| 29|         4| 20000|
|  Paul| 24|         3| 20000|
|Hailey| 21|         1| 15000|
| Shein| 23|         2| 18000|
+------+---+----------+------+



In [5]:
#criando uma função pra definir senioridade
def seniority(years):
  if 0 <= years < 4:
    seniority = 'junior' 
  elif 4 <= years < 8:
    seniority = 'pleno'
  elif years >= 8  :
    seniority = 'senior'
  return seniority

In [6]:
from pyspark.sql.functions import col, udf
reg = udf(lambda q: seniority(q))

In [7]:
#criando uma coluna de senioridade
df_pyspark.withColumn('seniority',reg(col("experience"))).show()

+------+---+----------+------+---------+
|  Name|age|experience|salary|seniority|
+------+---+----------+------+---------+
|  Kate| 31|        10| 30000|   senior|
| Susan| 30|         8| 25000|   senior|
|   Sun| 29|         4| 20000|    pleno|
|  Paul| 24|         3| 20000|   junior|
|Hailey| 21|         1| 15000|   junior|
| Shein| 23|         2| 18000|   junior|
+------+---+----------+------+---------+



In [8]:
df_pyspark = df_pyspark.withColumn('seniority',reg(col("experience")))

In [10]:
#groupby
# aggregate .agg  .avg  .sum .max .mean .min

df_pyspark.groupBy('seniority').sum().show()

+---------+--------+---------------+-----------+
|seniority|sum(age)|sum(experience)|sum(salary)|
+---------+--------+---------------+-----------+
|    pleno|      29|              4|      20000|
|   junior|      68|              6|      53000|
|   senior|      61|             18|      55000|
+---------+--------+---------------+-----------+



In [14]:
#groupby
# aggregate .agg  .avg  .sum .max .mean .min

df_pyspark.groupBy('seniority').mean().show()

+---------+------------------+---------------+------------------+
|seniority|          avg(age)|avg(experience)|       avg(salary)|
+---------+------------------+---------------+------------------+
|    pleno|              29.0|            4.0|           20000.0|
|   junior|22.666666666666668|            2.0|17666.666666666668|
|   senior|              30.5|            9.0|           27500.0|
+---------+------------------+---------------+------------------+



In [16]:
#ordenando   .desc() ou .asc()
df_pyspark.orderBy(df_pyspark.salary.asc()).show()

+------+---+----------+------+---------+
|  Name|age|experience|salary|seniority|
+------+---+----------+------+---------+
|Hailey| 21|         1| 15000|   junior|
| Shein| 23|         2| 18000|   junior|
|   Sun| 29|         4| 20000|    pleno|
|  Paul| 24|         3| 20000|   junior|
| Susan| 30|         8| 25000|   senior|
|  Kate| 31|        10| 30000|   senior|
+------+---+----------+------+---------+

